In [132]:
import dask.dataframe as dd
from dask.dataframe import from_pandas
from dask.dataframe.utils import make_meta
from neo4j import GraphDatabase
from neo4j.exceptions import ClientError
from dask.distributed import Client, LocalCluster, get_worker
import dask

import os
import time
from tqdm import tqdm
import pandas as pd
import re
import gc
import numpy as np

#importers
from pm4py import convert_to_event_log, convert_to_dataframe, format_dataframe

# Miners
from pm4py import convert_to_petri_net, serialize, deserialize
from pm4py import discover_dfg as dfg_discovery

from pm4py.algo.discovery.alpha import algorithm as alpha_miner
from pm4py.algo.discovery.heuristics import algorithm as heuristics_miner
from pm4py.algo.discovery.inductive import algorithm as inductive_miner
from pm4py.algo.discovery.correlation_mining import algorithm as correlation_miner
from pm4py.algo.discovery.temporal_profile import algorithm as temporal_profile_discovery


# Evaluators
from pm4py.algo.evaluation.simplicity import algorithm as simplicity_evaluator
from pm4py.algo.evaluation.replay_fitness import algorithm as replay_fitness_evaluator
from pm4py.algo.evaluation.precision import algorithm as precision_evaluator
from pm4py.algo.evaluation.generalization import algorithm as generalization_evaluator

In [2]:
dask.config.set({'distributed.scheduler.active-memory-manager.start': True})

In [3]:
import sys
sys.setrecursionlimit(30000)

In [4]:
import ctypes

def trim_memory() -> int:
    libc = ctypes.CDLL("libc.so.6")
    return libc.malloc_trim(0)

In [5]:
class graph_driver():
    def __init__(self, uri_scheme='bolt', host='localhost', port='7687', username='neo4j', password='123456'):
        self.uri_scheme = uri_scheme
        self.host = host
        self.port = port
        
        self.username = username
        self.password = password
        
        self.connection_uri = "{uri_scheme}://{host}:{port}".format(uri_scheme=self.uri_scheme, host=self.host, port=self.port)
        self.auth = (self.username, self.password)
        self.driver = GraphDatabase.driver(self.connection_uri, auth=self.auth)
        
    def __del__(self):
        self._close_driver()
    
    def _close_driver(self):
        if self.driver:
            self.driver.close()
    
    def run_single_query(self, query):
        res = None
        with self.driver.session() as session:
            raw_res = session.run(query)
            res = self.format_raw_res(raw_res)
        return res
    
    def run_bulk_query(self, query_list):
        results = []
        with self.driver.session() as session:
            for query in tqdm(query_list):
                raw_res = session.run(query)
                res = self.format_raw_res(raw_res)
                results.append({'query':query, 'result':res})
        return results
    
    def reset_graph(self, db=None):
        return self.run_single_query("MATCH (n) DETACH DELETE n")
    
    def test_connection(self):
        return self.run_single_query("MATCH (n) RETURN COUNT(n) as nodes")
    
    @staticmethod
    def format_raw_res(raw_res):
        res = []
        for r in raw_res:
            res.append(r)
        return res

In [6]:
def useExecutionTime(func):
    
    def compute(*args, **kwargs):
        begin = time.time()
        
        result = func(*args, **kwargs)
        
        end = time.time()
        
        return {"result": result, "execution_time": end - begin}
 
    return compute

@useExecutionTime
def getComputeTime(*args, **kwargs):
    return dask.compute(*args, **kwargs)

In [7]:
cluster = LocalCluster(n_workers=2, threads_per_worker=1, memory_limit='3GiB')
client = Client(cluster)

In [8]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 2,Total memory: 6.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44553,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 2
Started: Just now,Total memory: 6.00 GiB
Comm: tcp://127.0.0.1:43723,Total threads: 1
Dashboard: http://127.0.0.1:33919/status,Memory: 3.00 GiB
Nanny: tcp://127.0.0.1:45319,


In [9]:
cluster.adapt(minimum=2, maximum=5)

In [10]:
def run_gc(dask_worker,**kwargs):
    gc.collect()
    return True

# Register the GC function as a plugin
client.register_worker_plugin(run_gc, "my_gc_plugin")

{'tcp://127.0.0.1:43723': {'status': 'OK'},
 'tcp://127.0.0.1:44433': {'status': 'OK'}}

In [163]:
columnTypes = {
    'case:IDofConceptCase': 'string',
    'case:Includes_subCases': 'string',
    'case:Responsible_actor': 'string',
    'case:caseProcedure': 'string',
    'case:concept:name': 'int64',
    'dueDate': 'string',
    'case:termName': 'string',
    'dateStop': 'string'
}

# list of file paths to be loaded
file_paths = ['BPIC15_1.csv', 'BPIC15_2.csv']

# load the first file as a Dask dataframe
df = dd.read_csv(file_paths[0], dtype=columnTypes)

# iterate over the remaining files
for file_path in file_paths[1:]:
    # usecols parameter to load only the columns that are present in both dataframes
    df_temp = dd.read_csv(file_path)
    # concatenate the dataframes along the rows
    df = dd.concat([df, dd.read_csv(file_path, dtype=columnTypes)])

# df = df.drop('case:endDatePlanned', axis=1)
    
# fileName = 'BPIC15_*'
# df = dd.read_csv('{fileName}.csv'.format(fileName=fileName), dtype=columnTypes)
for column in df.columns:
    if re.search("[Dd]ate.*|time.*", column):
        df[column] = dask.dataframe.to_datetime(df[column], utc=True)

df = df.drop('case:endDatePlanned', axis=1)

In [165]:
def getDFGQueries(dfg, start_activities, end_activities):
    listOfQueries = []
    queryTemplate = """
        MERGE (p:Activity {{name: '{parent}'}})
        MERGE (c:Activity {{name: '{child}'}})
        MERGE (p)-[r:PRODUCES]->(c)
        ON CREATE SET r.frequency={frequency}
        ON MATCH SET r.frequency=r.frequency+{frequency}
    """
    for parent, child in dfg:
        frequency = dfg[(parent, child)]
        template = queryTemplate.format(parent=parent, child=child, frequency=frequency)
        listOfQueries.append(template)
        
    queryTemplateSA = """
        MERGE (p:StartActivity {{name: '{activity}'}})
        ON CREATE SET p.frequency={frequency}
        ON MATCH SET p.frequency=p.frequency+{frequency}
    """
    queryTemplateEA = """
        MERGE (p:EndActivity {{name: '{activity}'}})
        ON CREATE SET p.frequency={frequency}
        ON MATCH SET p.frequency=p.frequency+{frequency}
    """
    
    for activity, frequency in start_activities.items():
        template = queryTemplateSA.format(activity=activity, frequency=frequency)
        listOfQueries.append(template)
        
    for activity, frequency in end_activities.items():
        template = queryTemplateEA.format(activity=activity, frequency=frequency)
        listOfQueries.append(template)
    
    
    return listOfQueries

In [166]:
def transformToDFG(dfgResult):
    result = {}
    for record in dfgResult:
        result[(record["parent"], record["child"])] = record["frequency"]
    
    return result

def transformToStartEndActivity(activities):
    result = {}
    for record in activities:
        result[record['name']] = record["frequency"]
        
    return result

In [167]:
def getDFG():
    queries = {
        "dfgQuery": """MATCH result=(p:Activity)-[r:PRODUCES]->(c:Activity) RETURN p.name as parent, c.name as child, r.frequency as frequency""",
        "startEndActivitiesQuery": ["MATCH (a:StartActivity) RETURN a.name as name , a.frequency as frequency", "MATCH (a:EndActivity) RETURN a.name as name , a.frequency as frequency"],
    }
    
    neo4jConnection = graph_driver(uri_scheme="neo4j",host="neo4j", password="123456")
    
    dfgResult = neo4jConnection.run_single_query(queries['dfgQuery'])
    startEndActivitiesResult = neo4jConnection.run_bulk_query(queries['startEndActivitiesQuery'])
    return [transformToDFG(dfgResult), transformToStartEndActivity(startEndActivitiesResult[0]["result"]), transformToStartEndActivity(startEndActivitiesResult[1]["result"])]
    

In [169]:
indexed_df = df.set_index('case:concept:name', drop=False, sorted=True)

In [53]:
indexed_df.index = indexed_df.index.rename('caseId')
indexed_df = indexed_df.repartition(npartitions=4)

In [54]:
dfg, start, end = getDFG()

100% 2/2 [00:00<00:00, 240.40it/s]


In [55]:
@useExecutionTime
def getMinerResult(dfg, miner, threshold = 0.5):
    result = {}
    if miner == 'heuristic_miner':
        net, im, fm = heuristics_miner.apply_dfg(dfg, parameters={heuristics_miner.Variants.CLASSIC.value.Parameters.DEPENDENCY_THRESH: threshold})
    elif miner == 'inductive_miner':
        net, im, fm = inductive_miner.apply_dfg(dfg)
    elif miner == 'alpha_miner':
        net, im, fm = alpha_miner.apply_dfg(dfg)
    
    result[miner] = serialize(net, im, fm)
    
    return result
    
def setLazyMiners(dfg):
    lazyList = []
    miners = [
        'heuristic_miner',
        'inductive_miner',
#         'alpha_miner'
    ]
    for miner in miners:
        task = dask.delayed(getMinerResult)(dfg, miner)
        lazyList.append(task)
    
    return lazyList

In [56]:
@useExecutionTime
def getMetrics(log, miner, metric, net, im, fm):
    result = {
        miner: {
            metric: 0
        }
    }
    if metric == 'fitness':
        result[miner][metric] = replay_fitness_evaluator.apply(log, net, im, fm, variant=replay_fitness_evaluator.TOKEN_BASED)
    elif metric == 'simplicity':
        result[miner][metric] = simplicity_evaluator.apply(net)
    elif metric == 'precision':
        result[miner][metric] = precision_evaluator.apply(log, net, im, fm, variant=precision_evaluator.Variants.ETCONFORMANCE_TOKEN)
    elif metric == 'generalization':
        result[miner][metric] = generalization_evaluator.apply(log, net, im, fm)
    
    return result

def setLazyMetrics(log, miners):
    lazyList = []
    metrics = [
        'fitness',
        'simplicity',
        'precision', 
        'generalization'
    ]
    
    for metric in metrics:
        for miner in miners:
            algorithm = list(miner['result'].keys())[0]
            net, im, fm = deserialize(miner['result'][algorithm])
            task = dask.delayed(getMetrics)(log, algorithm, metric, net, im, fm)
            lazyList.append(task)
    
    return lazyList

In [57]:
lazyMiners = setLazyMiners(dfg)

In [58]:
lazyMinersResults = dask.compute(*lazyMiners)

In [59]:
lazyMetrics = setLazyMetrics(indexed_df, lazyMinersResults)

In [60]:
lazyMetricsResults = dask.compute(*lazyMetrics)

/opt/conda/lib/python3.9/site-packages/dask/dataframe/io/csv.py:182: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
replaying log with TBR, completed variants ::  34%|███▍      | 25124/74049 [04:34<09:55, 82.15it/s]2023-02-18 17:16:42,644 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 2.10 GiB -- Worker memory limit: 3.00 GiB
2023-02-18 17:16:42,655 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 2.10 GiB -- Worker memory limit: 3.00 

replaying log with TBR, completed variants ::  34%|███▍      | 25303/74049 [04:36<08:23, 96.76it/s]2023-02-18 17:16:44,510 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 2.10 GiB -- Worker memory limit: 3.00 GiB
2023-02-18 17:16:44,526 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 2.10 GiB -- Worker memory limit: 3.00 GiB
replaying log with TBR, completed variants ::  34%|███▍      | 25353/74049 [04:36<08:23, 96.73it/s]2023-02-18 17:16:45,075 - distributed.worker_memory - WARNING - Unmanaged memory use is high. Th

replaying log with TBR, completed variants ::  34%|███▍      | 25517/74049 [04:38<09:56, 81.41it/s]2023-02-18 17:16:46,995 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 2.11 GiB -- Worker memory limit: 3.00 GiB
2023-02-18 17:16:47,019 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 2.11 GiB -- Worker memory limit: 3.00 GiB
replaying log with TBR, completed variants ::  34%|███▍      | 25534/74049 [04:39<10:49, 74.65it/s]2023-02-18 17:16:47,233 - distributed.worker_memory - WARNING - Unmanaged memory use is high. Th

replaying log with TBR, completed variants ::  35%|███▍      | 25691/74049 [04:40<09:20, 86.21it/s]2023-02-18 17:16:49,068 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 2.11 GiB -- Worker memory limit: 3.00 GiB
2023-02-18 17:16:49,125 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 2.11 GiB -- Worker memory limit: 3.00 GiB
replaying log with TBR, completed variants ::  35%|███▍      | 25727/74049 [04:41<09:38, 83.54it/s]2023-02-18 17:16:49,567 - distributed.worker_memory - WARNING - Unmanaged memory use is high. Th

replaying log with TBR, completed variants ::  35%|███▍      | 25792/74049 [04:42<11:41, 68.77it/s]2023-02-18 17:16:50,513 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 2.11 GiB -- Worker memory limit: 3.00 GiB
2023-02-18 17:16:50,519 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 2.11 GiB -- Worker memory limit: 3.00 GiB
replaying log with TBR, completed variants ::  35%|███▍      | 25872/74049 [04:43<11:25, 70.30it/s]2023-02-18 17:16:51,557 - distributed.worker_memory - WARNING - Unmanaged memory use is high. Th

...

replaying log with TBR, completed variants ::  90%|████████▉ | 66516/74049 [12:05<01:16, 98.31it/s]2023-02-18 17:24:13,498 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 2.84 GiB -- Worker memory limit: 3.00 GiB
2023-02-18 17:24:13,530 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 2.84 GiB -- Worker memory limit: 3.00 GiB
replaying log with TBR, completed variants ::  90%|████████▉ | 66566/74049 [12:05<01:17, 96.65it/s]2023-02-18 17:24:14,034 - distributed.worker_memory - WARNING - Unmanaged memory use is high. Th

replaying log with TBR, completed variants ::  90%|█████████ | 66709/74049 [12:07<01:13, 99.45it/s]2023-02-18 17:24:15,505 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 2.84 GiB -- Worker memory limit: 3.00 GiB
2023-02-18 17:24:15,528 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 2.84 GiB -- Worker memory limit: 3.00 GiB
replaying log with TBR, completed variants ::  69%|██████▉   | 51316/74049 [02:36<01:11, 316.04it/s]2023-02-18 17:24:16,504 - distributed.worker_memory - WARNING - Unmanaged memory use is high. T

replaying log with TBR, completed variants ::  70%|██████▉   | 51650/74049 [02:37<01:08, 326.23it/s]2023-02-18 17:24:17,528 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 2.85 GiB -- Worker memory limit: 3.00 GiB
2023-02-18 17:24:17,534 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 2.85 GiB -- Worker memory limit: 3.00 GiB
replaying log with TBR, completed variants ::  90%|█████████ | 66957/74049 [12:09<01:11, 98.52it/s]2023-02-18 17:24:18,022 - distributed.worker_memory - WARNING - Unmanaged memory use is high. T

replaying log with TBR, completed variants ::  91%|█████████ | 67133/74049 [12:11<01:29, 77.44it/s]2023-02-18 17:24:20,037 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 2.85 GiB -- Worker memory limit: 3.00 GiB
2023-02-18 17:24:20,064 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 2.85 GiB -- Worker memory limit: 3.00 GiB
replaying log with TBR, completed variants ::  91%|█████████ | 67162/74049 [12:12<01:17, 88.32it/s]2023-02-18 17:24:20,341 - distributed.worker_memory - WARNING - Unmanaged memory use is high. Th

KeyboardInterrupt: 

In [ ]:
import io, json
with io.open('{fileName}_results.txt'.format(fileName=fileName), 'w', encoding='utf-8') as f:
    f.write(json.dumps(lazyMetricsResults, ensure_ascii=False))

In [ ]:
client.close()